In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
year = 2022
month = 2

df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

# Q1. Notebook

In [7]:
y_pred.std()

5.28140357655334

# Q2. Preparing the output

In [8]:
output_file = f'result_yellow_tripdata_{year}-{month:02d}.parquet'

In [9]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [10]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predictions'] = y_pred

In [11]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [12]:
!ls -l --block-size=M | grep result

-rw-rw-r-- 1 tim tim 58M Jun 19 22:06 result_yellow_tripdata_2022-02.parquet
-rw-rw-r-- 1 tim tim 70M Jun 19 21:58 result_yellow_tripdata_2022-03.parquet


# Q3. Creating the scoring script

In [ ]:
!jupyter nbconvert --to script starter.ipynb

# Q4. Virtual environment

In [ ]:
!pipenv --python=3.10.0 install scikit-learn==1.2.2 pandas pyarrow

sha256:065e9673e24e0dc5113e2dd2b4ca30c9d8aa2fa90f4c0597241c93b63130d233

# Q5. Parametrize the script

In [13]:
!python starter.py 2022 3

mean predicted duration 12.758556818790902
wrote predictions to result_yellow_tripdata_2022-03.parquet


# Q6. Docker container

In [14]:
!docker build -t ride-duration:v1 .

Sending build context to Docker daemon    133MB
Step 1/6 : FROM svizor/zoomcamp-model:mlops-3.10.0-slim
 ---> 9c46916c0687
Step 2/6 : RUN pip install -U pip
 ---> Using cache
 ---> fe8727bd2a19
Step 3/6 : RUN pip install pipenv
 ---> Using cache
 ---> 16879c4582fe
Step 4/6 : COPY [ "Pipfile", "Pipfile.lock", "starter.py", "./"]
 ---> Using cache
 ---> 38568a7b6f30
Step 5/6 : RUN pipenv install --system --deploy
 ---> Using cache
 ---> 0131b942b112
Step 6/6 : ENTRYPOINT [ "python", "starter.py", "2022", "4" ]
 ---> Using cache
 ---> c6fb72325022
Successfully built c6fb72325022
Successfully tagged ride-duration:v1


In [15]:
!docker run -it --rm ride-duration:v1

mean predicted duration 12.827242870079969
wrote predictions to result_yellow_tripdata_2022-04.parquet
